In [1]:
import numpy as np
from matplotlib import pyplot as plt
from random import randint
import random
import copy
import csv
import time
from __future__ import division 
from scipy.spatial import distance
from nbvd import NBVD

import sys
sys.path.insert(0, "/Users/kty/Documents/Projeto-Experimentos/dataSetSintetic/")
from SyntheticData import SyntheticData

In [2]:
#Cuidado con los valores quando K e L = a
params = {
    'k' : {
        'a' : 2, 'b' : 3, 'd' : 3, 'e': 3, 'f': 4, 'g': 4, 'h': 5, 'i': 4
    },
    'l' : {
        'a' : 2, 'b' : 3, 'd' : 3, 'e': 3, 'f': 4, 'g': 4, 'h': 5, 'i': 4
    },
    'c' : {
        'a' : 0, 'b' : 1, 'd' : 1, 'e': 1, 'f': 2, 'g': 2, 'h': 3, 'i': 2, 'c': 0
    }
}
lst_num_params = [[2, 3], [2, 3, 4, 5, 6, 7], [2, 3, 4, 5, 6, 7, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]] #[3, 6, 9]


In [ ]:
def run_nbvd():
    address_file = '/Users/kty/Documents/Projeto-Experimentos/Testes-NBVD/Sintetic_data/Experimento_v1/'
    data_format_option = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'] #
    lst_sparsity = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 98]

    #initial content
    with open(address_file + 'nBVD_sintetic-data.csv','w') as f:
        f.write('Nome,Erro,Tempo\n') # TRAILING NEWLINE

    escrita = []    
    #leer la data
    for option in data_format_option:
        print "------------------------ Iterando dados: %s ---------------------" % option
        data_orig,  x_row, y_col = SyntheticData().generate_dataset(option, noise=1, noise_background=False, shuffle=False,  esparsidad=False, niv_spar = 0)
        rows_orig, columns_orig = data_orig.shape
        file_pc = address_file + 'normal/Matriz/'
        nome = str(option) + '-structure'
        SyntheticData().save_data(data_orig, file_pc, nome)
        SyntheticData().graph_save_data(data_orig, file_pc, nome)
        
        #Guardando matriz sen la rotulagem de linhas e colunas    
        X = data_orig[0:rows_orig-1][:,0:columns_orig-1]
        n, m = X.shape
        nome = str(option) + '-structure_X'
        SyntheticData().save_data(X, file_pc, nome)
        SyntheticData().graph_save_data(X, file_pc, nome)

        np.savetxt(address_file + 'normal/Matriz_fmt/' + nome + '-fmt.csv', X, fmt="%.5f", delimiter=',')
        
        lst_num_cogrupos = lst_num_params[params['c'][option]]
        print "lst_num_cogrupos %s " % lst_num_cogrupos

        for k in lst_num_cogrupos:
            for l in lst_num_cogrupos:
                print '-----------estrutura: %s  (k = %s, l = %s) -----------' %(option,k,l)
                
                error = np.inf
                inicio = time.time()
                for i in xrange(1):
                    print "Iteration: %s" % i 
                    U_t, S_t, V_t, rows_ind_t, cols_ind_t, error_t, U_norm_t, V_norm_t = NBVD().matrix_factorization_clustering(X,
                                                                                         k, l, norm=False, num_iters=1000)
                    
                    U = U_t
                    S = S_t
                    V = V_t
                    rows_ind = rows_ind_t
                    cols_ind = cols_ind_t
                    error = error_t
                    U_norm = U_norm_t
                    V_norm = V_norm_t
                fim = time.time()
                tempo = fim - inicio
                print"Tempo: %s seconds " % tempo
                nome = str(option) + '_' + str(k) + 'x' + str(l) # a_2x2_S
                
                #Save U, S, V e U, S, V normalizado
                #Matriz U
                np.savetxt(address_file + 'normal/U-S-V/' + nome + '_U.csv', U, delimiter=',')
                np.savetxt(address_file + 'normal/U-S-V_fmt/' + nome + '_U-fmt.csv', U, fmt="%.5f ", delimiter=',')

                np.savetxt(address_file + 'normalizado/U-S-V/' + nome + '_U-norm.csv', U_norm, delimiter=',')
                np.savetxt(address_file + 'normalizado/U-S-V_fmt/' + nome + '_U-norm-fmt.csv', U_norm, fmt="%.5f ", delimiter=',')

                #Matriz S
                np.savetxt(address_file + 'normal/U-S-V/' + nome + '_S.csv', S, delimiter=',')
                np.savetxt(address_file + 'normal/U-S-V_fmt/' + nome + '_S-fmt.csv', S, fmt="%.5f ", delimiter=',')

                np.savetxt(address_file + 'normalizado/U-S-V/' + nome + '_S.csv', S, delimiter=',')
                np.savetxt(address_file + 'normalizado/U-S-V_fmt/' + nome + '_S-fmt.csv', S, fmt="%.5f ", delimiter=',')

                #Matriz V
                np.savetxt(address_file + 'normal/U-S-V/' + nome + '_V.csv', V, delimiter=',')
                np.savetxt(address_file + 'normal/U-S-V_fmt/' + nome + '_V-fmt.csv', V, fmt="%.5f ", delimiter=',')

                np.savetxt(address_file + 'normalizado/U-S-V/' + nome + '_V-norm.csv', V_norm, delimiter=',')
                np.savetxt(address_file + 'normalizado/U-S-V_fmt/' + nome + '_V-norm-fmt.csv', V_norm, fmt="%.5f ", delimiter=',')

                reconstruction = U.dot(S).dot(V.T)
                reconstruction_norm = U_norm.dot(S).dot(V_norm.T)
                np.savetxt(address_file + 'normal/U-S-V/' + nome + '_reconstruction.csv', reconstruction, delimiter=',')
                np.savetxt(address_file + 'normal/U-S-V_fmt/' + nome + '_reconstruction-fmt.csv', reconstruction, fmt="%.5f ", delimiter=',')

                np.savetxt(address_file + 'normalizado/U-S-V/' + nome + '_reconstruction-norm.csv', reconstruction_norm, delimiter=',')
                np.savetxt(address_file + 'normalizado/U-S-V_fmt/' + nome + '_reconstruction-norm-fmt.csv', reconstruction_norm, fmt="%.5f ", delimiter=',')

                # Geração de graficos
                #plt.matshow(U.T, cmap=plt.cm.Blues)
                plt.matshow(U, cmap=plt.cm.Blues)
                plt.title('NBVD U result')
                plt.grid()
                plt.savefig(address_file + 'normal/Img_U-S-V/%s_U.png' % nome)
                plt.cla
                plt.close()

                plt.matshow(S, cmap=plt.cm.Blues)
                plt.title('NBVD S result')
                plt.grid()
                plt.savefig(address_file + 'normal/Img_U-S-V/%s_S.png' % nome)
                #plt.show()
                plt.cla
                plt.close()

                plt.matshow(V.T, cmap=plt.cm.Blues)
                plt.title('NBVD V.T result')
                plt.grid()
                plt.savefig(address_file + 'normal/Img_U-S-V/%s_V.png' % nome)
                plt.cla
                plt.close()

                plt.matshow(reconstruction, cmap=plt.cm.Blues)
                plt.title('NBVD reconstruction result')
                plt.tick_params(labeltop=False, labelbottom=False, labelright=False, labelleft=False)
                plt.grid()
                plt.savefig(address_file + 'normal/Img_U-S-V/%s_reconstruction.png' % nome)
                plt.cla
                plt.close()

                plt.matshow(reconstruction_norm, cmap=plt.cm.Blues)
                plt.title('NBVD reconstruction norm result')
                plt.tick_params(labeltop=False, labelbottom=False, labelright=False, labelleft=False)
                plt.grid()
                plt.savefig(address_file + 'normal/Img_U-S-V/%s_reconstruction-norm.png' % nome)
                plt.cla
                plt.close()
                
                escrita = []
                escrita.append(nome)
                escrita.append(error)
                escrita.append(tempo)
                #print escrita
                with open(address_file + 'nBVD_sintetic-data.csv','a') as f:
                    writer=csv.writer(f)
                    writer.writerow(escrita)
                print 'Error: %s' % error
                print '-----------------------------------------------'

    return 0